In [ ]:
spark.stop()

hello


In [18]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [19]:


spark = SparkSession.builder \
    .appName("DataCo - Prédiction Retards") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.default.parallelism", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print("Spark initialisé ✔️")

df_test = spark.range(10)
df_test.show()



ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
feature_numeric_cols = [
    "Days for shipment (scheduled)",
    "Benefit per order",
    "Sales per customer",
    "Order Item Discount",
    "Order Item Discount Rate",
    "Order Item Product Price",
    "Order Item Profit Ratio",
    "Order Item Quantity",
    "Sales",
    "Order Profit Per Order"
]

feature_categorical_cols = [
    "Type",
    "Shipping Mode",
    "Market",
    "Customer Segment",
    "Order State",
    "Order Region",
    "Category Name"
]


cible = ["Late_delivery_risk"]  # Variable cible

In [5]:
from pyspark.sql.functions import col

base_cols = feature_numeric_cols + feature_categorical_cols + cible

df = (
    spark.read
        .option("header", True)
        .option("inferSchema", True)
        .csv("../data/DataCoSupplyChainDataset.csv")
        .filter(col("Delivery Status") != "Shipping canceled")
        .select(*base_cols)
)

df.explain(True)


== Parsed Logical Plan ==
'Project ['Days for shipment (scheduled), 'Benefit per order, 'Sales per customer, 'Order Item Discount, 'Order Item Discount Rate, 'Order Item Product Price, 'Order Item Profit Ratio, 'Order Item Quantity, 'Sales, 'Order Profit Per Order, 'Type, 'Shipping Mode, 'Market, 'Customer Segment, 'Order State, 'Order Region, 'Category Name, 'Late_delivery_risk]
+- Filter NOT (Delivery Status#29 = Shipping canceled)
   +- Relation [Type#24,Days for shipping (real)#25,Days for shipment (scheduled)#26,Benefit per order#27,Sales per customer#28,Delivery Status#29,Late_delivery_risk#30,Category Id#31,Category Name#32,Customer City#33,Customer Country#34,Customer Email#35,Customer Fname#36,Customer Id#37,Customer Lname#38,Customer Password#39,Customer Segment#40,Customer State#41,Customer Street#42,Customer Zipcode#43,Department Id#44,Department Name#45,Latitude#46,Longitude#47,... 29 more fields] csv

== Analyzed Logical Plan ==
Days for shipment (scheduled): int, Benefit

In [6]:
df.printSchema()



root
 |-- Days for shipment (scheduled): integer (nullable = true)
 |-- Benefit per order: double (nullable = true)
 |-- Sales per customer: double (nullable = true)
 |-- Order Item Discount: double (nullable = true)
 |-- Order Item Discount Rate: double (nullable = true)
 |-- Order Item Product Price: double (nullable = true)
 |-- Order Item Profit Ratio: double (nullable = true)
 |-- Order Item Quantity: integer (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Order Profit Per Order: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Shipping Mode: string (nullable = true)
 |-- Market: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Order State: string (nullable = true)
 |-- Order Region: string (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Late_delivery_risk: integer (nullable = true)



In [7]:
df_verifier_equilibre = df.groupBy("Late_delivery_risk").count()
df_verifier_equilibre.show()

+------------------+-----+
|Late_delivery_risk|count|
+------------------+-----+
|                 1|98977|
|                 0|73788|
+------------------+-----+



In [8]:
from pyspark.sql.functions import col, when, count
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

# VOS FEATURES
feature_numeric_cols = [
    "Days for shipment (scheduled)", "Benefit per order", "Sales per customer",
    "Order Item Discount", "Order Item Discount Rate", "Order Item Product Price",
    "Order Item Profit Ratio", "Order Item Quantity", "Sales", "Order Profit Per Order"
]

feature_categorical_cols = [
    "Type", "Shipping Mode", "Market", "Customer Segment",
    "Order State", "Order Region", "Category Name"
]

cible = ["Late_delivery_risk"]

# Charger les données
base_cols = feature_numeric_cols + feature_categorical_cols + cible

df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("../data/DataCoSupplyChainDataset.csv")
    .filter(col("Delivery Status") != "Shipping canceled")
    .select(*base_cols)
)


# 1. GESTION DES VALEURS MANQUANTES
print("=== Vérification des valeurs manquantes ===")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

df_clean = df.dropna()
print(f"Lignes avant: {df.count()}, Lignes après: {df_clean.count()}")


# 2. ENCODAGE DES VARIABLES CATÉGORIELLES
print("\n=== Encodage des variables catégorielles ===")
indexers = []
for col_name in feature_categorical_cols:
    indexer = StringIndexer(
        inputCol=col_name, 
        outputCol=col_name + "_indexed", 
        handleInvalid="keep"
    )
    indexers.append(indexer)

pipeline_encoding = Pipeline(stages=indexers)
model_encoding = pipeline_encoding.fit(df_clean)
df_encoded = model_encoding.transform(df_clean)

encoded_cols = [c + "_indexed" for c in feature_categorical_cols]


# 3. NORMALISATION DES FEATURES NUMÉRIQUES
print("\n=== Normalisation des features numériques ===")
assembler_num = VectorAssembler(
    inputCols=feature_numeric_cols, 
    outputCol="numeric_features",
    handleInvalid="keep"
)
df_assembled = assembler_num.transform(df_encoded)

scaler = StandardScaler(
    inputCol="numeric_features", 
    outputCol="numeric_features_scaled",
    withMean=True, 
    withStd=True
)
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)


# 4. PRÉPARATION FINALE POUR LE MODÈLE
print("\n=== Assembly final ===")
all_feature_cols = ["numeric_features_scaled"] + encoded_cols
assembler_final = VectorAssembler(
    inputCols=all_feature_cols, 
    outputCol="features",
    handleInvalid="keep"
)
df_final = assembler_final.transform(df_scaled)

# Dataset prêt
df_ready = df_final.select("features", col("Late_delivery_risk").alias("label"))

print("\n=== Dataset prêt pour le modèle ===")
print(f"Nombre total de lignes: {df_ready.count()}")
df_ready.show(5, truncate=False)

# Distribution finale
print("\n=== Distribution de la variable cible ===")
df_ready.groupBy("label").count().show()

=== Vérification des valeurs manquantes ===
+-----------------------------+-----------------+------------------+-------------------+------------------------+------------------------+-----------------------+-------------------+-----+----------------------+----+-------------+------+----------------+-----------+------------+-------------+------------------+
|Days for shipment (scheduled)|Benefit per order|Sales per customer|Order Item Discount|Order Item Discount Rate|Order Item Product Price|Order Item Profit Ratio|Order Item Quantity|Sales|Order Profit Per Order|Type|Shipping Mode|Market|Customer Segment|Order State|Order Region|Category Name|Late_delivery_risk|
+-----------------------------+-----------------+------------------+-------------------+------------------------+------------------------+-----------------------+-------------------+-----+----------------------+----+-------------+------+----------------+-----------+------------+-------------+------------------+
|                

## Pipline

In [10]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import  GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import col

feature_numeric_cols = [
    "Days for shipment (scheduled)", "Benefit per order", "Sales per customer",
    "Order Item Discount", "Order Item Discount Rate", "Order Item Product Price",
    "Order Item Profit Ratio", "Order Item Quantity", "Sales", "Order Profit Per Order"
]

feature_categorical_cols = [
    "Type", "Shipping Mode", "Market", "Customer Segment",
    "Order State", "Order Region", "Category Name"
]

df = (
    spark.read.option("header", True).option("inferSchema", True)
    .csv("../data/DataCoSupplyChainDataset.csv")
    .filter(col("Delivery Status") != "Shipping canceled")
    .select(*(feature_numeric_cols + feature_categorical_cols + ["Late_delivery_risk"]))
).dropna()

indexers = [StringIndexer(inputCol=c, outputCol=c+"_indexed", handleInvalid="keep")
            for c in feature_categorical_cols]

assembler_num = VectorAssembler(
    inputCols=feature_numeric_cols,
    outputCol="num_features"
)

scaler = StandardScaler(
    inputCol="num_features",
    outputCol="num_features_scaled",
    withMean=True,
    withStd=True
)

assembler_final = VectorAssembler(
    inputCols=["num_features_scaled"] + [c+"_indexed" for c in feature_categorical_cols],
    outputCol="features"
)

rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="Late_delivery_risk",
    maxBins=2000
)

pipeline = Pipeline(stages=indexers + [assembler_num, scaler, assembler_final, rf])

train, test = df.randomSplit([0.8, 0.2], seed=42)

model = pipeline.fit(train)
pred = model.transform(test)

evaluator = BinaryClassificationEvaluator(
    labelCol="Late_delivery_risk",
    rawPredictionCol="rawPrediction"
)

print("AUC =", evaluator.evaluate(pred))


gbt = GBTClassifier(
    featuresCol="features",
    labelCol="Late_delivery_risk",
    maxIter=50,
    maxDepth=5,
    maxBins=2000
)


AUC = 0.751825774386722


In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import col

# VOS FEATURES
feature_numeric_cols = [
    "Days for shipment (scheduled)", "Benefit per order", "Sales per customer",
    "Order Item Discount", "Order Item Discount Rate", "Order Item Product Price",
    "Order Item Profit Ratio", "Order Item Quantity", "Sales", "Order Profit Per Order"
]

feature_categorical_cols = [
    "Type", "Shipping Mode", "Market", "Customer Segment",
    "Order State", "Order Region", "Category Name"
]

cible = ["Late_delivery_risk"]

# Charger les données
base_cols = feature_numeric_cols + feature_categorical_cols + cible

df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("../data/DataCoSupplyChainDataset.csv")
    .filter(col("Delivery Status") != "Shipping canceled")
    .select(*base_cols)
)

# Nettoyer les données
df_clean = df.dropna()
print(f"Dataset nettoyé: {df_clean.count()} lignes")

# Vérifier le nombre de valeurs uniques par colonne catégorielle
print("\n=== Nombre de valeurs uniques par variable catégorielle ===")
for col_name in feature_categorical_cols:
    nb_unique = df_clean.select(col_name).distinct().count()
    print(f"{col_name}: {nb_unique} valeurs uniques")


# ====================================================================
# ÉTAPES DE PRÉTRAITEMENT
# ====================================================================

# 1. Encodage des variables catégorielles avec StringIndexer
indexers = []
for col_name in feature_categorical_cols:
    indexer = StringIndexer(
        inputCol=col_name, 
        outputCol=col_name + "_indexed", 
        handleInvalid="keep"
    )
    indexers.append(indexer)

indexed_cols = [c + "_indexed" for c in feature_categorical_cols]

# 2. OneHotEncoding pour les modèles qui en ont besoin (Neural Network)
encoders = []
for col_name in feature_categorical_cols:
    encoder = OneHotEncoder(
        inputCol=col_name + "_indexed",
        outputCol=col_name + "_encoded"
    )
    encoders.append(encoder)

encoded_cols = [c + "_encoded" for c in feature_categorical_cols]

# 3. Assembly des features numériques
assembler_num = VectorAssembler(
    inputCols=feature_numeric_cols, 
    outputCol="numeric_features",
    handleInvalid="keep"
)

# 4. Normalisation
scaler = StandardScaler(
    inputCol="numeric_features", 
    outputCol="numeric_features_scaled",
    withMean=True, 
    withStd=True
)

# 5. Assembly final pour Random Forest et GBT (avec indexed)
assembler_final_trees = VectorAssembler(
    inputCols=["numeric_features_scaled"] + indexed_cols, 
    outputCol="features",
    handleInvalid="keep"
)

# 6. Assembly final pour Neural Network (avec OneHot)
assembler_final_mlp = VectorAssembler(
    inputCols=["numeric_features_scaled"] + encoded_cols, 
    outputCol="features",
    handleInvalid="keep"
)


# ====================================================================
# MODÈLES NON-LINÉAIRES
# ====================================================================

# MODÈLE 1 : RANDOM FOREST
# SOLUTION : Augmenter maxBins pour gérer plus de catégories
rf = RandomForestClassifier(
    featuresCol="features", 
    labelCol="Late_delivery_risk",
    numTrees=100,
    maxDepth=10,
    maxBins=2000,  # ← AUGMENTÉ pour gérer Order State (1081 valeurs)
    minInstancesPerNode=1,
    seed=42
)

# MODÈLE 2 : GRADIENT BOOSTING TREES
gbt = GBTClassifier(
    featuresCol="features", 
    labelCol="Late_delivery_risk",
    maxIter=100,
    maxDepth=5,
    maxBins=2000,  # ← AUGMENTÉ également
    stepSize=0.1,
    seed=42
)

# MODÈLE 3 : NEURAL NETWORK
# Pour MLP, on a besoin de compter les features après OneHotEncoding
# On va le calculer dynamiquement après le preprocessing
mlp = MultilayerPerceptronClassifier(
    featuresCol="features",
    labelCol="Late_delivery_risk",
    maxIter=100,
    blockSize=128,
    seed=42
)


# ====================================================================
# SPLIT TRAIN/TEST
# ====================================================================

train_data, test_data = df_clean.randomSplit([0.8, 0.2], seed=42)

print(f"\nDonnées d'entraînement: {train_data.count()} lignes")
print(f"Données de test: {test_data.count()} lignes")


# ====================================================================
# FONCTION POUR ENTRAÎNER ET ÉVALUER UN MODÈLE
# ====================================================================

def entrainer_et_evaluer(model, nom_modele, use_onehot=False):
    print(f"\n{'='*60}")
    print(f"MODÈLE : {nom_modele}")
    print(f"{'='*60}")
    
    # Choisir le bon assembler
    if use_onehot:
        assembler_final = assembler_final_mlp
        stages = [*indexers, *encoders, assembler_num, scaler, assembler_final, model]
    else:
        assembler_final = assembler_final_trees
        stages = [*indexers, assembler_num, scaler, assembler_final, model]
    
    # Créer le pipeline
    pipeline = Pipeline(stages=stages)
    
    # Entraîner
    print(f"⏳ Entraînement en cours...")
    model_trained = pipeline.fit(train_data)
    print(f"✅ Entraînement terminé !")
    
    # Prédictions
    predictions_train = model_trained.transform(train_data)
    predictions_test = model_trained.transform(test_data)
    
    # Evaluateurs
    evaluator_auc = BinaryClassificationEvaluator(
        labelCol="Late_delivery_risk",
        rawPredictionCol="rawPrediction",
        metricName="areaUnderROC"
    )
    
    evaluator_accuracy = MulticlassClassificationEvaluator(
        labelCol="Late_delivery_risk",
        predictionCol="prediction",
        metricName="accuracy"
    )
    
    evaluator_f1 = MulticlassClassificationEvaluator(
        labelCol="Late_delivery_risk",
        predictionCol="prediction",
        metricName="f1"
    )
    
    evaluator_precision = MulticlassClassificationEvaluator(
        labelCol="Late_delivery_risk",
        predictionCol="prediction",
        metricName="weightedPrecision"
    )
    
    evaluator_recall = MulticlassClassificationEvaluator(
        labelCol="Late_delivery_risk",
        predictionCol="prediction",
        metricName="weightedRecall"
    )
    
    # Calculer les métriques
    auc_train = evaluator_auc.evaluate(predictions_train)
    auc_test = evaluator_auc.evaluate(predictions_test)
    
    accuracy_train = evaluator_accuracy.evaluate(predictions_train)
    accuracy_test = evaluator_accuracy.evaluate(predictions_test)
    
    f1_train = evaluator_f1.evaluate(predictions_train)
    f1_test = evaluator_f1.evaluate(predictions_test)
    
    precision_test = evaluator_precision.evaluate(predictions_test)
    recall_test = evaluator_recall.evaluate(predictions_test)
    
    # Afficher les résultats
    print(f"\n📊 RÉSULTATS - {nom_modele}")
    print(f"{'-'*60}")
    print(f"TRAIN | AUC: {auc_train:.4f} | Accuracy: {accuracy_train:.4f} | F1: {f1_train:.4f}")
    print(f"TEST  | AUC: {auc_test:.4f} | Accuracy: {accuracy_test:.4f} | F1: {f1_test:.4f}")
    print(f"TEST  | Precision: {precision_test:.4f} | Recall: {recall_test:.4f}")
    
    # Matrice de confusion
    print(f"\n📈 Matrice de confusion - {nom_modele}")
    confusion_matrix = predictions_test.groupBy("Late_delivery_risk", "prediction").count()
    confusion_matrix.orderBy("Late_delivery_risk", "prediction").show()
    
    return {
        'modele': nom_modele,
        'auc_test': auc_test,
        'accuracy_test': accuracy_test,
        'f1_test': f1_test,
        'precision_test': precision_test,
        'recall_test': recall_test,
        'pipeline': model_trained
    }


# ====================================================================
# ENTRAÎNER ET COMPARER TOUS LES MODÈLES NON-LINÉAIRES
# ====================================================================

resultats = []

# Random Forest
# resultats.append(entrainer_et_evaluer(rf, "Random Forest", use_onehot=False))

# Gradient Boosting
resultats.append(entrainer_et_evaluer(gbt, "Gradient Boosting Trees", use_onehot=False))

# Neural Network (avec OneHot encoding)
# D'abord, configurer les layers correctement
# On va le faire après avoir compté les features
print("\n⏳ Configuration du Neural Network...")

# Créer un pipeline temporaire pour compter les features
temp_pipeline = Pipeline(stages=[*indexers, *encoders, assembler_num, scaler, assembler_final_mlp])
temp_model = temp_pipeline.fit(train_data.limit(100))  # Juste pour compter
temp_data = temp_model.transform(train_data.limit(1))
nb_features = temp_data.select("features").first()[0].size

print(f"Nombre de features après preprocessing: {nb_features}")

# Configurer le MLP avec le bon nombre de features
layers = [nb_features, 128, 64, 2]  # 2 classes
mlp.setLayers(layers)

resultats.append(entrainer_et_evaluer(mlp, "Neural Network (MLP)", use_onehot=True))


# ====================================================================
# COMPARAISON FINALE
# ====================================================================

print(f"\n{'='*80}")
print("COMPARAISON DES MODÈLES NON-LINÉAIRES")
print(f"{'='*80}")
print(f"{'Modèle':<30} {'AUC':<10} {'Accuracy':<10} {'F1-Score':<10} {'Precision':<10} {'Recall':<10}")
print(f"{'-'*80}")

for r in resultats:
    print(f"{r['modele']:<30} {r['auc_test']:<10.4f} {r['accuracy_test']:<10.4f} {r['f1_test']:<10.4f} {r['precision_test']:<10.4f} {r['recall_test']:<10.4f}")

# Trouver le meilleur modèle
meilleur = max(resultats, key=lambda x: x['f1_test'])
print(f"\n🏆 MEILLEUR MODÈLE : {meilleur['modele']} (F1-Score: {meilleur['f1_test']:.4f})")
print(f"{'='*80}")

ConnectionRefusedError: [Errno 111] Connection refused